In [1]:
#importing functions 
import numpy as np
import matplotlib.pyplot as plt
import pycce as pc 
import pandas as pd
from scipy.optimize import curve_fit 
import mpi4py

/Users/markoneill/Desktop/Physics/SS/Capstone/VScode Capstone Folder/Capstone_environment/lib/python3.10/site-packages/numba/core/decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [2]:
#define function to converge parameters
def runner(variable, values):
    print('Running:' + str(variable))
    invalue = parameters[variable]
    calc = pc.Simulator(spin=cen, bath=atoms, **parameters)
    ls = []
    
    for idx, v in enumerate(values):
        print("Progress: " + str(idx/len(values)*100) + "%")
        setattr(calc, variable, v)
        l = calc.compute(ts, method='cce',
                         nbstates=nb,
                         quantity='coherence',
                         parallel = True,)
        
        
        #print('a run done') # uncomment to see progress
        ls.append(l.real)

    parameters[variable] = invalue
    ls = pd.DataFrame(ls, columns=ts, index=values).T
    return ls

In [ ]:
#import xyz file
uc = pd.read_csv('VOTPP_opt.xyz', skiprows=2, header=None, delimiter='      ', engine='python') #enter specific directory
#seperate columns into numpy arrays
N = np.array(uc[0])
x = np.array(uc[1])
y = np.array(uc[2])
z = np.array(uc[3])
#set up unit cell
sic = pc.BathCell(13.2613, 13.2613, 9.6851, 90, 90, 90, 'deg') #used optimized structure note there is also an x-ray structure with slightly different values
# z axis in cell coordinates
sic.zdir = [0, 0, 1]

#populate cell
for i in range(len(N)):
    sic.add_atoms((N[i], [x[i], y[i], z[i]]), type='angstrom')

#assign position of qubit 
pos1 = x[76], y[76], z[76]
qpos = sic.to_cell(pos1)

#generate supercell - nuclear bath 
cell=100
atoms = sic.gen_supercell(cell, seed = 8000) #left fixed for convergence tests to avoid changes
#set          spin | gyro | quadrupole 
spin_types = [('C', 1 / 2, 6.72828),    
              ('H', 1 / 2, 26.7522),
              ('N', 1,     1.9331, 20.44 )
              #('V', 7/2, 7.05, -350)  # not added for consistency between tests
              ]   
atoms.add_type(*spin_types)

#set up the center 
cen = pc.CenterArray(size=1, position=qpos, 
                     spin=7/2, D=-350, gyro=-7.05,  
                     alpha=[0,0,0,0,0,1,0,0], beta=[0,0,0,0,1,0,0,0]) #set to -3/2 to -1/2

#parameters
nb=25
ts = np.linspace(0, 7e-2, 201) #expected T2 of 3.3e-2

#dictionary of parameters
parameters = dict(
    order=2, # CCE order
    r_bath=20,  # Size of the bath in A
    r_dipole=10,  # Cutoff of pairwise clusters in A
    pulses = 1, # N pulses in CPMG sequence
    magnetic_field=[0,0,10000] #set to 1T
) 
